In [1]:
from typing import Dict, List, Any
import pandas as pd

from doraemon import Doraemon
from sketch_of_thought import SoT
from reasoning_path_builder import ReasoningPath
from prompt_template_collection import PromptTemplate

df=pd.read_pickle('/kaggle/input/building-hotpotqa-injected-shuffled-dataset/hotpotqa_injected_shuffled_all_in_one_df.pkl')
df.shape

2025-06-23 08:55:01.709841: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750668901.968533      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750668902.039547      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(1300, 5)

In [2]:
logger=Doraemon.get_logger(name=__name__, logfile='sot_on_hotpotqa_injected')

df.rename(columns={'answer':'ground_truth'}, inplace=True)


df['evidence_injected'] = df['evidence_injected'].apply(lambda x: ' '.join(x))
df['evidence_type']='Injected'

In [3]:
df['question'] = df.apply(lambda row: f"{row['question']} \nContext: {row['evidence_injected']}", axis=1)
logger.info(df.iloc[0].question)

2025-06-23 08:55:15,844 INFO VIVA Media AG changed it's name in 2004. What does their new acronym stand for? 
Context: VIVA Media GmbH (until 2004 "VIVA Media AG") is a music television network originating from Germany. This is irrelevant content


In [4]:
def final_prompt(question:str) -> str:
    return SoT.few_short_support(
        task_name='HOTPOTQA', 
        prompt=SoT.get_initialized_prompt(paradigm='conceptual_chaining'),
        few_short=SoT.get_few_short(task_name='HOTPOTQA'),
        question=question)

df['prompt']=df['question'].apply(final_prompt)

In [5]:
tasks: List[Dict[str, Any]]=df.head(1200).to_dict(orient='records')

temperatures = [i * 0.25 for i in range(9)]

final_tasks=[]
for t in tasks:
    for tp in temperatures:
        final_tasks.append((t, tp))

logger.info(len(final_tasks))

2025-06-23 08:55:15,892 INFO 10800


In [6]:
D:Dict[str, str] = PromptTemplate.sot_template_hotpotqa()
logger.info(len(D))

results:List[Dict]=ReasoningPath.run_parallel_inference(final_tasks, D, logger, 'sots', None)

2025-06-23 08:55:15,905 INFO 2
  4%|▍         | 414/10800 [04:40<1:54:34,  1.51it/s]2025-06-23 08:59:57,809 ERROR Error processing quetion Flower Alley was bred by the trainer who was killed at what Fayette County, Kentucky airport? 
Context:  He was bred at Bona Terra Farms by George Brunacini, who was killed in the August 27, 2006, crash of Comair Flight 5191 at Blue Grass Airport in Lexington, Kentucky. This is irrelevant content with exception expected string or bytes-like object, got 'NoneType'
  4%|▍         | 415/10800 [04:40<1:56:12,  1.49it/s]2025-06-23 08:59:58,466 ERROR Error processing quetion Flower Alley was bred by the trainer who was killed at what Fayette County, Kentucky airport? 
Context:  He was bred at Bona Terra Farms by George Brunacini, who was killed in the August 27, 2006, crash of Comair Flight 5191 at Blue Grass Airport in Lexington, Kentucky. This is irrelevant content with exception expected string or bytes-like object, got 'NoneType'
  4%|▍         | 416/

In [7]:
logger.info(results[0])

2025-06-23 13:35:41,810 INFO {'question': 'VIVA Media AG changed it\'s name in 2004. What does their new acronym stand for? \nContext: VIVA Media GmbH (until 2004 "VIVA Media AG") is a music television network originating from Germany. This is irrelevant content', 'context': 'VIVA Media GmbH (until 2004 "VIVA Media AG") is a music television network originating from Germany. This is irrelevant content', 'r_s': '<think>\nLet’s think through this step by step\n#VIVA Media AG → changed name in 2004\n#New name → new acronym\n</think>\n\\boxed{This question is fact-based recall}', 'answer': 'This question is fact-based recall', 'ground_truth': 'Gesellschaft mit beschränkter Haftung', 'tokens': 45, 'temperature': 0.0}


In [8]:
filtered_results = [r for r in results if r is not None]

pd.DataFrame(filtered_results).to_pickle('sots_df.pkl')